In [1]:
req = {
    "instanceOf": {
        "value": "https://bibliokeia.com/works/26"
    },
    "adminMetadata": {
        "status": {
            "value": "http://id.loc.gov/vocabulary/mstatus/n",
            "label": "Novo"
        },
        "descriptionConventions": {
            "value": "http://id.loc.gov/vocabulary/descriptionConventions/aacr",
            "label": "AACr"
        }
    },
    "identifiedBy": {
        "isbn": "978-85-211-0193-2"
    },
    "resource": [
        {
            "type": {
                "value": "Instance",
                "label": "Instância"
            }
        },
        {
            "type": {
                "value": "Print",
                "label": "Impresso"
            }
        }
    ],
    "title": {
        "mainTitle": "Destruição e Conservação da Floresta Amazônica"
    },
    "physicalDetails": {
        "carrier": {
            "value": "http://id.loc.gov/vocabulary/carriers/nc",
            "label": "Volume"
        },
        "extent": "215 p.",
        "issuance": {
            "value": "http://id.loc.gov/vocabulary/issuance/mono",
            "label": "Unidade única"
        },
        "media": {
            "value": "http://id.loc.gov/vocabulary/mediaTypes/n",
            "label": "Não mediado"
        }
    },
    "provisionActivity": [
        {
            "place": {
                "base": "bk",
                "value": "https://bibliokeia.com/authorities/139",
                "label": "Manaus"
            },
            "agent": {
                "base": "bk",
                "value": "https://bibliokeia.com/authorities/141",
                "label": "Editora INPA"
            },
            "date": "2022"
        }
    ],
    "image": {
        "cover": "https://acdn.mitiendanube.com/stores/023/619/products/destruicao-e-conservacao-da-floresta-amazonica-11-ded0b44703723270e316751900084128-480-0.webp"
    }
}

In [3]:
from datetime import datetime
from src.schemas.catalog.instance import BfInstance


request = BfInstance(**req)
now = datetime.now()
request.adminMetadata.creationDate = now
request.adminMetadata.identifiedBy = 'TESTE'

In [5]:
request.identifiedBy

IdentifiedBy(local=None, isbn='978-85-211-0193-2')

In [2]:
from rdflib import Graph
graph = Graph()
graph.parse('http://id.loc.gov/resources/instances/14562666.rdf')
graph.serialize('Instance.ttl')

<Graph identifier=Neccf3a8f10644e5cab4350bffdaec236 (<class 'rdflib.graph.Graph'>)>